In [ ]:
import os
import time
import pandas as pd
from google import genai
from google.genai import types
import gspread
from gspread import utils


# =============================================================================
# 1. CONFIGURACIÓN DE RUTAS Y API
# =============================================================================


# Rutas de entrada local (solo para la imagen y el prompt .txt)
PROMPT_FILE_PATH = r"C:\Users\roy.ruiz\Downloads\prompt_api.txt"
IMAGE_DIRECTORY = r"C:\Users\roy.ruiz\Pictures\CategoriaBebes"


# Configuración de Google Sheets (¡VERIFICADO!)
OAUTH_CREDENTIALS_FILE = 'credentials.json'
SPREADSHEET_ID = '1cMBpwYzeKj_d2-2Cd_RX40BiPR_qzUob9bO9KG-6AbA'
WORKSHEET_NAME = 'Hoja1'


# Inicializar el cliente de Gemini
try:
   client = genai.Client()
except Exception as e:
   print(f"ERROR CRÍTICO: Falló la inicialización del cliente de Gemini. ¿Está GEMINI_API_KEY configurada?: {e}")
   exit()




# =============================================================================
# 2. FUNCIONES DE AYUDA (Gemini)
# =============================================================================


def load_prompt(file_path: str) -> str:
   """Carga el texto del prompt desde un archivo."""
   try:
       with open(file_path, 'r', encoding='utf-8') as f:
           return f.read().strip()
   except Exception:
       with open(file_path, 'r', encoding='latin-1') as f:
           return f.read().strip()




def process_product_with_gemini(image_full_path: str, prompt_text: str) -> str:
   """Llama a la API de Gemini con imagen y prompt, implementando reintentos."""


   if not os.path.exists(image_full_path):
       return f"ERROR_IMAGEN: Archivo no encontrado en {image_full_path}"


   mime_type = "image/jpeg"
   if image_full_path.lower().endswith(('.png', '.PNG')):
       mime_type = "image/png"


   try:
       with open(image_full_path, "rb") as f:
           image_bytes = f.read()
   except Exception as e:
       return f"ERROR_LECTURA: Falló al leer la imagen: {e}"


   # Sintaxis de Part.from_text corregida y modelo seguro usado.
   contents = [
       types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
       types.Part.from_text(text=prompt_text)
   ]


   MAX_RETRIES = 5
   base_delay = 5


   for attempt in range(MAX_RETRIES):
       try:
           # Modelo compatible
           response = client.models.generate_content(
               model='gemini-2.5-flash',
               contents=contents
           )
           return response.text.strip().replace('\n', ' ')


       except Exception as e:
           error_message = str(e)


           if attempt < MAX_RETRIES - 1:
               sleep_time = base_delay * (2 ** attempt)
               time.sleep(sleep_time)
           else:
               return f"ERROR_API_FATAL: Fallo después de 5 intentos. Último error: {error_message}"


   return "ERROR_INESPERADO: Bucle de reintento fallido."




# =============================================================================
# 3. FLUJO PRINCIPAL DE EJECUCIÓN CON GOOGLE SHEETS
# =============================================================================


def run_automation():
   print("--- INICIANDO AUTOMATIZACIÓN DE ATRIBUTOS CON GOOGLE SHEETS ---")


   # === SOLUCIÓN AL ERROR DE RUTA (CRÍTICA) ===
   # Esta parte asume que el script se ejecuta en un entorno donde se manejan credenciales
   # Si las credenciales son manejadas por la variable de entorno GSPREAD_OAUTH_CREDENTIALS,
   # el path local puede no ser estrictamente necesario aquí.
   script_dir = os.path.dirname(os.path.abspath(__file__))
   absolute_credentials_path = os.path.join(script_dir, OAUTH_CREDENTIALS_FILE)
   # ==========================================


   # 1. AUTENTICACIÓN Y CONEXIÓN
   try:
       # gspread usa la variable de entorno para autenticación OAUTH
       gc = gspread.oauth()
       spreadsheet = gc.open_by_key(SPREADSHEET_ID)
       worksheet = spreadsheet.worksheet(WORKSHEET_NAME)
   except Exception as e:
       print(f"ERROR FATAL DE AUTENTICACIÓN O CONEXIÓN A SHEETS: {e}")
       print(
           "\nVerifica la configuración: Asegúrate de que las credenciales están configuradas correctamente.")
       return


   print(f"✅ Conexión exitosa a la hoja: {WORKSHEET_NAME}")


   prompt = load_prompt(PROMPT_FILE_PATH)
   if not prompt:
       print(f"ERROR: No se pudo cargar el prompt desde {PROMPT_FILE_PATH}")
       return


   # 2. Lógica de Carga y Reanudación desde Sheets
   try:
       all_values = worksheet.get_all_values()
   except Exception as e:
       print(f"ERROR al leer los datos de la hoja. Asegúrate de que tienes conexión y permisos. Error: {e}")
       return


   if len(all_values) <= 1:
       print("ERROR: La hoja de cálculo está vacía o solo tiene encabezados.")
       return


   header = all_values[0]
   data_rows = all_values[1:]


   # Determinar las columnas por el encabezado (insensible a mayúsculas/minúsculas)
   try:
       header_lower = [h.strip().lower() for h in header]


       idx_id = header_lower.index("id")
       idx_image = header_lower.index("image")
       idx_attributes = header_lower.index("gemini_attributes")
   except ValueError as e:
       print(f"ERROR CRÍTICO: Los encabezados ('ID', 'image', 'Gemini_Attributes') no fueron encontrados en la hoja.")
       print(f"Encabezados encontrados: {header}")
       return


   # 💡 SOLUCIÓN AL ERROR: Usar rowcol_to_a1 para determinar la letra de la columna
   # +1 porque el índice de la columna es base 0, y gspread es base 1.
   col_a1_notation = utils.rowcol_to_a1(1, idx_attributes + 1)
   attributes_col_letter = col_a1_notation.rstrip('1')


   print(f"Total de productos a procesar: {len(data_rows)}")
   print(f"Columna de atributos para la escritura individual: {attributes_col_letter}")


   # 3. Procesar cada registro y GUARDAR EN CADA ITERACIÓN


   products_processed = 0


   for i, row in enumerate(data_rows):
       # Número de fila real en Sheets (i + 2: Fila 1 es encabezado, i es base 0)
       row_num = i + 2


       # Aseguramos que la columna de atributos exista para leer
       current_row = list(row)
       if len(current_row) <= idx_attributes:
           # Si la fila es más corta, la tratamos como vacía en atributos
           attributes_value = ""
       else:
           attributes_value = current_row[idx_attributes].strip()


       sku_id = current_row[idx_id].strip()
       image_filename = current_row[idx_image].strip()


       # LÓGICA DE SALTO
       # Saltamos si ya está procesado y no es un error API fatal
       if attributes_value != "" and not attributes_value.startswith('ERROR_API_FATAL'):
           print(f"⏩ Saltando fila {row_num}: ID {sku_id} (Ya procesado).")
           continue


       products_processed += 1


       # Construir la ruta completa de la imagen
       image_full_path = os.path.join(IMAGE_DIRECTORY, image_filename)


       print(f"⚙️ Procesando fila {row_num}/{len(data_rows) + 1}: ID {sku_id}...")


       # Llamar a la función principal de Gemini con reintentos
       attributes = process_product_with_gemini(image_full_path, prompt)


       # =================================================================
       # GUARDADO CRÍTICO: ACTUALIZACIÓN INDIVIDUAL DE CELDA
       # =================================================================
       cell_to_update = f'{attributes_col_letter}{row_num}'


       try:
           # Actualiza UNA SOLA celda
           worksheet.update_acell(cell_to_update, attributes)


           if attributes.startswith("ERROR"):
               print(f"   ❌ Fallo: {attributes}. Guardado en celda {cell_to_update}.")
           else:
               print(f"   ✔️ OK. Guardado en celda {cell_to_update}. Atributos (50 carácteres): {attributes[:50]}...")


           # Pausa para evitar el límite de escritura de Google Sheets (crucial)
           time.sleep(1.5)


       except Exception as e:
           print(f"ERROR al guardar en la celda {cell_to_update} de Sheets: {e}")
           time.sleep(5)  # Pausa más larga si hay un error de conexión/API.
       # =================================================================


   # 4. Finalización
   if products_processed > 0:
       print(f"\n✨ Proceso Completado. Se procesaron y guardaron {products_processed} productos individualmente. ✨")
   else:
       print("\nTodos los registros ya estaban procesados o la hoja está vacía.")




if __name__ == "__main__":
   run_automation()